In [ ]:
import torch
import sys,os
import matplotlib.pyplot as plt
import numpy as np
from torch import nn
from torch.nn import functional as Func
from importlib import reload
import loupe_env.line_sampler
from loupe_env.line_sampler import *
from loupe_env.loupe_wrap import *
import utils
from utils import kplot
from typing import List

from unet.unet_model import UNet

import torch.optim as optim

In [ ]:
reload(loupe_env.loupe_wrap)
from loupe_env.loupe_wrap import *

In [ ]:
reload(loupe_env.line_sampler)
from loupe_env.line_sampler import *

In [ ]:
shape = [320,320]
slope = 1
sparsity = .25
preselect = True
preselect_num = 24
sampler = LOUPESampler(shape, slope, sparsity, preselect, preselect_num)

### loupe run test

In [ ]:
%run loupe_train_v2.py -ngpu 1

### sampler test

In [ ]:
traindata  = np.load('/home/huangz78/data/traindata_x.npz')
dtyp       = torch.float
trainxfull = torch.tensor(traindata['xfull'],dtype=dtyp)
testdata   = np.load('/home/huangz78/data/testdata_x.npz')
testxfull  = torch.tensor(testdata['xfull'],dtype=dtyp)
# fullmask = torch.tensor(data['mask'])

In [ ]:
xstar = trainxfull[0:10,:,:].view(10,1,320,320)
ystar = torch.fft.fftn(xstar,dim=(2,3),norm='ortho')
# y = torch.fft.fftn(xfull[0:3,:,:],dim=(0,1),norm='ortho')
# y = torch.reshape(y,(y.shape[0],-1,y.shape[1],y.shape[2]))
print(ystar.shape)

In [ ]:
sampler = LOUPESampler(shape=shape,slope=slope,sparsity=sparsity,preselect=preselect,preselect_num=preselect_num)
sampler.eval()
x_recon, mask = sampler(ystar,sparsity)
kplot(mask[0,0,:,0].detach().numpy(),roll=True)

In [ ]:
vec = torch.

### LOUPE pipeline

In [ ]:
#     def loss(self, pred_dict, target_dict, loss_type):
#         """
#         Args:
#             pred_dict:
#                 output: reconstructed image from downsampled kspace measurement
#                 energy: negative entropy of the probability mask
#                 mask: the binazried sampling mask (used for visualization)
#             target_dict:
#                 target: original fully sampled image
#             meta:
#                 recon_weight: weight of reconstruction loss
#                 entropy_weight: weight of the entropy loss (to encourage exploration)
#         """       
#         if loss_type == 'l1':
#             reconstruction_loss = Func.l1_loss(pred, target, size_average=True) 
#         elif loss_type == 'ssim':
#             reconstruction_loss = -torch.mean(compute_ssim_torch(pred, target))
#         elif loss_type == 'psnr':
#             reconstruction_loss = - torch.mean(compute_psnr_torch(pred, target))
#         else:
#             raise NotImplementedError
#         loss = reconstruction_loss
#         return loss

In [ ]:
loupe = LOUPE(shape=shape,slope=slope,sparsity=sparsity,preselect_num=preselect_num)
# img = torch.randn((1,1,320,320))
model_path = '/home/huangz78/checkpoints/loupe_spar_0.25_base_24.pt'
checkpoint = torch.load(model_path)
loupe.load_state_dict(checkpoint['model_state_dict'])
print('Loupe model is loaded successfully from: ' + model_path )
loupe.eval()

out,mask = loupe(ystar)
# plt.imshow(out[0,0,:,:].detach().numpy())
kplot(mask[0,0,:,0].detach().numpy(),roll=True)

In [ ]:
out,mask = loupe(ystar)
# plt.imshow(out[0,0,:,:].detach().numpy())
kplot(mask[0,0,:,0].detach().numpy(),roll=True)

In [ ]:
loupe_hist_path = '/home/huangz78/checkpoints/loupe_spar_0.25_base_24_history.npz'
loupe_hist = np.load(loupe_hist_path)

plt.figure()
plt.plot(loupe_hist['loss_train'])
plt.title('training L1 loss')
plt.yscale('log')
plt.show()

plt.figure()
plt.plot(loupe_hist['loss_val'])
plt.title('validation L1 loss')
plt.show()

In [ ]:
torch.norm(out[0,0,:,:] - img[0,0,:,:],p=1)/torch.norm(img[0,0,:,:],p=1)

In [ ]:
for name, param in loupe.samplers[0].named_parameters():
    if param.requires_grad:
        print(name, param.data)

In [ ]:
loupeTrain(trainxfull[0:21],testxfull[0:10],batchsize_train=5,batchsize_val=5)

In [ ]:
def normalize_data(batch):
    for img in range(len(batch)):
        batch[img,:,:] /= torch.max(torch.abs(batch[img,:,:]))
    return batch

def loupeTrain(traindata,valdata,\
               slope=5, sparsity=.25, preselect=True, preselect_num=24,\
               unet_skip=True, n_channels=1,\
               lrm=1e-3, lru=1e-4, weight_decay=0, momentum=0,\
               epochs=1, batchsize_train=5, batchsize_val=2, count_start=(0,0),\
               modelpath=None,\
               save_cp=True):
    
    '''
    train and test data are assumed to have the shape [size, heg, wid]
    '''
    
    # load sampler
    shape   = traindata.shape[1:3]
    
    sampler = None
    # load unet
    UNET = UNet(n_channels=n_channels,n_classes=1,bilinear=(not unet_skip),skip=unet_skip)
    unetpath = '/home/huangz78/checkpoints/unet_1_' + str(unet_skip) + '.pth'
    checkpoint = torch.load(unetpath)
    UNET.load_state_dict(checkpoint['model_state_dict'])

    loupe = LOUPE(n_channels=n_channels,unet_skip=unet_skip,shape=shape,slope=slope,sparsity=sparsity,\
                  preselect=preselect,preselect_num=preselect_num,\
                  sampler=sampler,unet=UNET)
    if modelpath is not None:
        checkpoint = torch.load(modelpath)
        loupe.load_state_dict(checkpoint['model_state_dict'])
    loupe.train()
    
    # training
    train_loss = []; val_loss = []
    epoch_count = count_start[0];  batchind = count_start[1]
    batch_nums  = int(np.ceil(traindata.shape[0]/batchsize_train))
    
    optimizer = optim.RMSprop([
                    {'params': loupe.samplers.parameters()},
                    {'params': loupe.unet.parameters(),'lr':lru}
                ], lr=lrm, weight_decay=weight_decay, momentum=momentum,eps=1e-10)
    criterion = nn.L1Loss()
    try:
        while epoch_count<epochs:
            while batchind<batch_nums:
                batch = np.arange(batchsize_train*batchind, min(batchsize_train*(batchind+1),traindata.shape[0]))
                databatch = normalize_data(traindata[batch,:,:]) if len(batch)>1 else normalize_data(traindata[batch,:,:].view(-1,shape[0],shape[1]))
                xstar = traindata[batch,:,:].view(batch.shape[0],-1,shape[0],shape[1])
                ystar = F.fftn(xstar,dim=(2,3),norm='ortho')
                x_recon = loupe(ystar)
                breakpoint()
                loss_train = criterion(x_recon,xstar)
                optimizer.zero_grad()
                loss_train.backward()
                optimizer.step()
                progress_str = f'[{epoch_count+1}/{epochs}][{min(batchsize_train*(batchind+1),traindata.shape[0])}/{traindata.shape[0]}]'
                print(progress_str + f' training loss: {loss_train.item()}')
                train_loss.append(loss_train.item())
                batchind += 1   

            # validation eval
            valbatchind = 0
            valbatch_nums = int(np.ceil(valdata.shape[0]/batchsize_val))
            loupe.eval()
            loss_val = 0
            while (valbatchind < valbatch_nums):
                batch = np.arange(valbatchind*batchind, min(batchsize_val*(valbatchind+1),valdata.shape[0]))
                databatch = normalize_data(valdata[batch,:,:]) if len(batch)>1 else normalize_data(valdata[batch,:,:].view(-1,shape[0],shape[1]))
                xstar = databatch.view(batch.shape[0],-1,shape[0],shape[1])
                ystar = F.fftn(xstar,dim=(2,3),norm='ortho')
                x_recon = loupe(ystar)
                loss_val += criterion(x_recon,xstar)
                valbatchind += 1
            progress_str = f'[{epoch_count+1}/{epochs}]'
            print('\n' + progress_str + f' validation loss: {loss_val.item()/valbatch_nums}')
            val_loss.append(loss_val.item()/valbatch_nums)
            loupe.train()

            # saving models
            if save_cp:
                dir_checkpoint = '/home/huangz78/checkpoints/'
                try:
                    os.mkdir(dir_checkpoint)
                    print('Created checkpoint directory')
                except OSError:
                    pass
                torch.save({'model_state_dict': loupe.state_dict()}, dir_checkpoint + 'loupe_model.pt')
                np.savez(dir_checkpoint+'loupe_history.npz',loss_train=train_loss, loss_val=val_loss)
                print(f'\t Checkpoint for Loupe saved after epoch {epoch_count + 1}!' + '\n')
            epoch_count += 1
            batchind = 0
    except KeyboardInterrupt: # need debug
        print('Keyboard Interrupted! Exit ~')
        torch.save({'model_state_dict': loupe.state_dict()}, dir_checkpoint + 'loupe_model.pt')
        np.savez(dir_checkpoint+'loupe_history.npz',loss_train=train_loss, loss_val=val_loss)
        print('Model is saved after keyboard interruption~')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)